In [1]:
import time
import glob
import os
import numpy as np
import pandas as pd
import cv2 as cv
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Flatten, Activation, Conv2D, MaxPool2D, Dense
from keras.optimizers import SGD, Adam
from tensorflow.keras import backend
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [2]:
path = './smear2005/database-pictures/normal_superficiel'

In [3]:
normal_images = []
segmented_images = []
shape_of_images = []

for r, d, f in os.walk(path):
    for filename in f:
        if('-d.bmp' in os.path.join(path, filename)):
            img = cv.imread(os.path.join(path, filename))
            rows, cols = img.shape[:2]
            shape_of_images.append([rows, cols])
            segmented_images.append(os.path.join(path, filename))
        else:
            normal_images.append(os.path.join(path, filename))

In [4]:
shape_of_images

[[234, 189],
 [168, 171],
 [280, 355],
 [300, 768],
 [393, 349],
 [242, 364],
 [406, 449],
 [364, 364],
 [382, 400],
 [316, 341],
 [261, 330],
 [289, 252],
 [182, 258],
 [331, 352],
 [267, 252],
 [127, 153],
 [145, 203],
 [187, 241],
 [238, 285],
 [132, 254],
 [173, 254],
 [229, 162],
 [284, 307],
 [344, 387],
 [313, 314],
 [289, 247],
 [276, 382],
 [310, 371],
 [288, 303],
 [334, 361],
 [282, 279],
 [326, 480],
 [382, 348],
 [314, 445],
 [259, 306],
 [239, 243],
 [226, 285],
 [239, 280],
 [255, 302],
 [277, 263],
 [252, 303],
 [358, 374],
 [292, 358],
 [337, 392],
 [276, 296],
 [284, 768],
 [318, 340],
 [353, 357],
 [381, 320],
 [267, 434],
 [335, 344],
 [431, 449],
 [439, 496],
 [389, 359],
 [191, 308],
 [206, 391],
 [280, 276],
 [311, 245],
 [418, 413],
 [349, 315],
 [324, 323],
 [354, 318],
 [345, 402],
 [382, 298],
 [310, 269],
 [399, 378],
 [310, 322],
 [336, 314],
 [357, 289],
 [331, 345],
 [280, 291],
 [362, 374],
 [319, 231],
 [297, 347]]

In [5]:
print(len(normal_images))
normal_images[0:5]

74


['./smear2005/database-pictures/normal_superficiel\\153958345-153958392-001.BMP',
 './smear2005/database-pictures/normal_superficiel\\153960256-153960295-002.BMP',
 './smear2005/database-pictures/normal_superficiel\\157181281-157181308-001.BMP',
 './smear2005/database-pictures/normal_superficiel\\157181387-157181418-001.BMP',
 './smear2005/database-pictures/normal_superficiel\\157181525-157181540-001.BMP']

In [6]:
print(len(segmented_images))
segmented_images[0:5]

74


['./smear2005/database-pictures/normal_superficiel\\153958345-153958392-001-d.bmp',
 './smear2005/database-pictures/normal_superficiel\\153960256-153960295-002-d.bmp',
 './smear2005/database-pictures/normal_superficiel\\157181281-157181308-001-d.bmp',
 './smear2005/database-pictures/normal_superficiel\\157181387-157181418-001-d.bmp',
 './smear2005/database-pictures/normal_superficiel\\157181525-157181540-001-d.bmp']

In [7]:
# img = cv.imread(normal_images[0])
# cv.imshow('image', img)
# cv.waitKey(0)

In [10]:
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model